# Using Mixup on TREC Dataset

## Import libraries

In [ ]:
from datasets import load_dataset # using 🤗 HugggingFace datasets library

## Load dataset

In [1]:
dataset = load_dataset("trec")

Downloading: 5.50kB [00:00, 2.27MB/s]                   
Downloading: 2.99kB [00:00, 1.26MB/s]                   
Using custom data configuration default


Downloading: 100%|██████████| 336k/336k [00:01<00:00, 260kB/s]
Downloading: 100%|██████████| 23.4k/23.4k [00:00<00:00, 73.9kB/s]
100%|██████████| 2/2 [00:00<00:00, 666.29it/s]


Dataset trec downloaded and prepared to C:\Users\ariha\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 38.31it/s]
